In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy import stats

import statsmodels.api as sm


import time
import re 

In [14]:

# def fast_linear_regression(x_arrays, y_arrays):
#     """
#     Compute slopes, intercepts, and their standard errors for n-point datasets.

#     This function uses a vectorized approach based on the analytical solutions
#     for ordinary least squares (OLS) regression parameters and their standard
#     errors.

#     params:
#         x_arrays (np.ndarray): A 2D NumPy array of x-coordinates, with shape
#             (n_datasets, n_points).
#         y_arrays (np.ndarray): A 2D NumPy array of y-coordinates, with shape
#             (n_datasets, n_points).

#     returns:
#         tuple: A tuple containing four 1D NumPy arrays:
#             - slopes (np.ndarray): The calculated slope for each dataset.
#             - intercepts (np.ndarray): The calculated intercept for each
#                 dataset.
#             - se_slopes (np.ndarray): The standard error of the slope for each
#                 dataset.
#             - se_intercepts (np.ndarray): The standard error of the intercept
#                 for each dataset.
#     """
    
#     # Ensure inputs are NumPy arrays
#     x = np.asarray(x_arrays)
#     y = np.asarray(y_arrays)

#     # Get the number of points per fit (n) and degrees of freedom (df)
#     n = x.shape[1]
#     df = n - 2
    
#     # --- Step 1: Calculate Slope and Intercept ---
#     x_mean = np.mean(x, axis=1)
#     y_mean = np.mean(y, axis=1)

#     # Sum of squares for x (Sxx) and sum of products for xy (Sxy)
#     ss_xx = np.sum((x - x_mean[:, np.newaxis])**2, axis=1)
#     ss_xy = np.sum((y - y_mean[:, np.newaxis]) * (x - x_mean[:, np.newaxis]), axis=1)

#     # Calculate slopes (b1)
#     # Handle the case where ss_xx is zero to avoid division by zero errors.
#     slopes = np.divide(ss_xy, ss_xx, out=np.full_like(ss_xy, np.nan), where=ss_xx!=0)
    
#     # Calculate intercepts (b0)
#     intercepts = y_mean - slopes * x_mean

#     # --- Step 2: Calculate Standard Errors ---
#     # Predicted y values and Residual Sum of Squares (RSS)
#     y_pred = intercepts[:, np.newaxis] + slopes[:, np.newaxis] * x
#     rss = np.sum((y - y_pred)**2, axis=1)

#     # Residual Standard Error (RSE)
#     if df > 0:
#         rse = np.sqrt(rss / df)
    
#         # Standard Error of the Slope (SE_b1)
#         se_slopes = rse / np.sqrt(ss_xx)
    
#         # Standard Error of the Intercept (SE_b0)
#         term_in_sqrt = (1/n) + (x_mean**2 / ss_xx)
#         se_intercepts = rse * np.sqrt(term_in_sqrt)
#     else:
#         se_slopes = np.nan*np.ones(len(slopes))
#         se_intercepts = np.nan*np.ones(len(slopes))

#     return slopes, intercepts, se_slopes, se_intercepts

# import numpy as np

# def fast_weighted_linear_regression(x_arrays, y_arrays, y_err_arrays):
#     """
#     Compute slopes, intercepts, and their standard errors for n-point datasets
#     using weighted least squares (WLS).

#     This function uses a vectorized approach based on the analytical solutions
#     for WLS regression parameters and their standard errors, assuming the
#     provided variances are known.

#     params:
#         x_arrays (np.ndarray): A 2D NumPy array of x-coordinates, with shape
#             (n_datasets, n_points).
#         y_arrays (np.ndarray): A 2D NumPy array of y-coordinates, with shape
#             (n_datasets, n_points).
#         y_err_arrays (np.ndarray): A 2D NumPy array of the variance (sigma^2)
#             of each y measurement, with shape (n_datasets, n_points).

#     returns:
#         tuple: A tuple containing four 1D NumPy arrays:
#             - slopes (np.ndarray): The calculated slope for each dataset.
#             - intercepts (np.ndarray): The calculated intercept for each dataset.
#             - se_slopes (np.ndarray): The standard error of the slope for each dataset.
#             - se_intercepts (np.ndarray): The standard error of the intercept for each dataset.
#     """
    
#     # --- Step 0: Data Preparation ---
#     x = np.asarray(x_arrays)
#     y = np.asarray(y_arrays)
#     y_err = np.asarray(y_err_arrays)
    
#     # If a single dataset is passed, add a dimension to make it 2D
#     if x.ndim == 1:
#         x = x[np.newaxis, :]
#         y = y[np.newaxis, :]
#         y_err = y_err[np.newaxis, :]

#     # Get the number of points per fit (n)
#     n_points = x.shape[1]
#     if n_points < 2:
#         nan_array = np.full(x.shape[0], np.nan)
#         return nan_array, nan_array, nan_array, nan_array

#     # --- Step 1: Calculate Weights and Weighted Sums ---
#     # The weight of each point is the inverse of its variance.
#     # Handle cases where variance is zero to avoid division errors.
#     weights = np.divide(1.0, y_err, out=np.zeros_like(y_err, dtype=float), where=y_err!=0)

#     # Calculate the necessary weighted sums for all datasets at once.
#     sw = np.sum(weights, axis=1)
#     swx = np.sum(weights * x, axis=1)
#     swy = np.sum(weights * y, axis=1)
#     swxx = np.sum(weights * x**2, axis=1)
#     swxy = np.sum(weights * x * y, axis=1)

#     # --- Step 2: Calculate Slope and Intercept ---
#     # This term is the determinant of the design matrix, used in several calculations.
#     delta = sw * swxx - swx**2

#     # Calculate slopes (b1) and intercepts (b0) for each dataset.
#     # Use np.divide to safely handle cases where delta is zero (e.g., all x are identical).
#     slopes = np.divide(sw * swxy - swx * swy, delta, out=np.full_like(delta, np.nan), where=delta!=0)
#     intercepts = np.divide(swxx * swy - swx * swxy, delta, out=np.full_like(delta, np.nan), where=delta!=0)

#     # --- Step 3: Calculate Standard Errors ---
#     # The standard errors are calculated assuming the input variances are known.
    
#     # Variance of the slope
#     var_slopes = np.divide(sw, delta, out=np.full_like(delta, np.nan), where=delta!=0)
#     se_slopes = np.sqrt(var_slopes)

#     # Variance of the intercept
#     var_intercepts = np.divide(swxx, delta, out=np.full_like(delta, np.nan), where=delta!=0)
#     se_intercepts = np.sqrt(var_intercepts)

#     return slopes, intercepts, se_slopes, se_intercepts


def glm_growth_rate_regression_with_offset(t_arrays,
                                           n_arrays,
                                           N_arrays,
                                           pop_size_arrays):
    """
    Compute growth rates (slopes) using a GLM with a population size offset.

    This function models the growth of the *absolute abundance* of a genotype,
    which is defined as frequency * population_size. It assumes the absolute
    abundance grows exponentially.

    The model fits ln(E[f] * pop_size) = b0 + b1*t, where f = n/N. This is
    achieved by using -ln(pop_size) as an offset in the GLM's linear predictor.

    params:
        t_arrays (np.ndarray): 2D array of time points, shape (n_datasets, n_points).
        n_arrays (np.ndarray): 2D array of observed counts (reads for a genotype),
            shape (n_datasets, n_points).
        N_arrays (np.ndarray): 2D array of total counts (total reads),
            shape (n_datasets, n_points).
        pop_size_arrays (np.ndarray): 2D array of the total population size
            (e.g., CFU/mL) at each time point, shape (n_datasets, n_points).

    returns:
        tuple: A tuple containing four 1D NumPy arrays:
            - slopes (np.ndarray): The calculated growth rate (k) for each dataset.
            - intercepts (np.ndarray): The calculated intercept for each dataset.
            - se_slopes (np.ndarray): The standard error of the growth rate.
            - se_intercepts (np.ndarray): The standard error of the intercept.
    """
    # --- Step 0: Data Preparation ---
    t = np.asarray(t_arrays)
    n = np.asarray(n_arrays)
    N = np.asarray(N_arrays)
    pop_size = np.asarray(pop_size_arrays)

    if t.ndim == 1:
        t = t[np.newaxis, :]
        n = n[np.newaxis, :]
        N = N[np.newaxis, :]
        pop_size = pop_size[np.newaxis, :]

    n_datasets = t.shape[0]

    # --- Step 1: Pre-allocate Result Arrays ---
    slopes = np.full(n_datasets, np.nan)
    intercepts = np.full(n_datasets, np.nan)
    se_slopes = np.full(n_datasets, np.nan)
    se_intercepts = np.full(n_datasets, np.nan)

    # --- Step 2: Loop and Fit GLM for Each Dataset ---
    for i in range(n_datasets):
        t_i = t[i, :]
        n_i = n[i, :]
        N_i = N[i, :]
        pop_size_i = pop_size[i, :]

        # --- Data Validation ---
        # Filter for points with valid sequencing and population data.
        valid_mask = (N_i > 0) & (n_i <= N_i) & (pop_size_i > 0)
        if np.sum(valid_mask) < 2:
            continue

        t_filt = t_i[valid_mask]
        n_filt = n_i[valid_mask]
        N_filt = N_i[valid_mask]
        pop_size_filt = pop_size_i[valid_mask]
        
        if np.all(n_filt == 0):
            continue

        # --- Prepare data for statsmodels ---
        endog = np.column_stack((n_filt, N_filt - n_filt))
        exog = sm.add_constant(t_filt)

        # --- Calculate the offset term ---
        # The offset is -ln(pop_size) to model ln(freq * pop_size).
        offset_term = -np.log(pop_size_filt)

        # --- Step 3: Fit the GLM with the offset ---
        try:
            glm_binom = sm.GLM(
                endog,
                exog,
                family=sm.families.Binomial(link=sm.families.links.Log()),
                offset=offset_term
            )
            results = glm_binom.fit()

            # --- Step 4: Store Results ---
            intercepts[i] = results.params[0]
            slopes[i] = results.params[1]
            se_intercepts[i] = results.bse[0]
            se_slopes[i] = results.bse[1]

        except Exception:
            continue

    return slopes, intercepts, se_slopes, se_intercepts


def get_growth_rates(obs_csv):
    
    print("Reading dataframe",flush=True)
    df = pd.read_csv(obs_csv)
    clean_df = df.loc[df["cfu_per_mL"] > 0,:].copy()
    clean_df.loc[:,"condition"] = (clean_df['genotype'].astype(str) + '_' +
                                   clean_df['selector'].astype(str) + '_' +
                                   clean_df['iptg'].astype(str))
    
    print("Grouping by genotype/selector/time",flush=True)
    count_df = clean_df.reset_index().groupby("condition", sort=False).agg(
        first_index=('index', 'first'),
        count=('condition', 'size'),
        max_count=('count','max'),
        max_freq=('freq','max'),
        max_cfu_per_mL=('cfu_per_mL','max')
    )

    return clean_df
        
    print("Creating output dataframe",flush=True)
    out_df = clean_df.loc[count_df["first_index"],["condition","genotype","selector","iptg"]]
    out_df["n"] = np.array(count_df["count"])
    out_df["max_count"] = np.array(count_df["max_count"])
    out_df["max_freq"] = np.array(count_df["max_freq"])
    out_df["max_cfu_per_mL"] = np.array(count_df["max_cfu_per_mL"])
    out_df["slope"] = np.nan
    out_df["slope_se"] = np.nan
    out_df["intercept"] = np.nan
    out_df["intercept_se"] = np.nan
    
    all_num_points_seen = np.unique(count_df["count"])
    all_num_points_seen.sort()
    all_num_points_seen = all_num_points_seen[::-1]
    
    for n in all_num_points_seen:
        if n > 1:
            print(f"fitting data with n = {n} non-zero time points.")
            conditions_to_grab = count_df.index[count_df["count"] == n]
            
            this_n_mask = clean_df["condition"].isin(conditions_to_grab)
            times = np.array(clean_df.loc[this_n_mask,"time"])
            times = times.reshape((times.shape[0]//n,n))
            
            ln_cfu = np.log(np.array(clean_df.loc[this_n_mask,"cfu_per_mL"]))
            ln_cfu = ln_cfu.reshape((ln_cfu.shape[0]//n,n))
    
            m, b, m_se, b_se = fast_linear_regression(times, ln_cfu)
    
            out_n_mask = out_df["condition"].isin(conditions_to_grab)
            out_df.loc[out_n_mask,"slope"] = m
            out_df.loc[out_n_mask,"slope_se"] = m_se
            out_df.loc[out_n_mask,"intercept"] = b
            out_df.loc[out_n_mask,"intercept_se"] = b_se

    out_df = out_df.drop(columns=["condition"])

    return out_df

#growth_df = get_growth_rates("many-state_wider/obs_many-state_molar_slower.csv")
            
obs_df = pd.read_csv("many-state_wider/obs_many-state_molar_slower.csv")


In [15]:
obs_df

,Unnamed: 0,genotype,selector,iptg,time,count,freq,cfu_per_mL
0,0,wt,kan,0.001,30,116709,3.071289e-03,947.349754
1,1,wt,kan,0.001,60,116465,3.064868e-03,949.136594
2,2,wt,kan,0.001,90,115579,3.041553e-03,945.775348
3,3,wt,kan,0.010,30,116806,3.073842e-03,948.958706
4,4,wt,kan,0.010,60,116305,3.060658e-03,949.492043
...,...,...,...,...,...,...,...,...
10794715,10794715,K59Y/M98Y,pheS,10.000,60,17,4.473684e-07,0.139967
10794716,10794716,K59Y/M98Y,pheS,10.000,90,11,2.894737e-07,0.093189
10794717,10794717,K59Y/M98Y,pheS,30.000,30,8,2.105263e-07,0.064108
10794718,10794718,K59Y/M98Y,pheS,30.000,60,13,3.421053e-07,0.105062


In [ ]:

def get_base_growth_rates(base_dir="."):
    """
    THIS IS A HACKED FUNCTION. 
    """
    
    base_growth_rates = {}
    
    for selector in ["kan","pheS"]:
    
        df = pd.read_csv(f"{base_dir}/{selector}_lib_file.csv")
        for i in tqdm(df.index):    
            clone = str(df.loc[i,"clone"]).strip()[1:-1].strip()

            if clone == "":
                clone = "wt"
            
            clone = str(re.sub("'","",clone))
            clone = clone.split(",")
            clone = [c.strip() for c in clone]
            clone = "/".join(clone)

            p0g = df.loc[i,f"{selector}_p0g"]
            p1g = df.loc[i,f"{selector}_p1g"]

            if clone not in base_growth_rates:
                base_growth_rates[clone] = {}

            obs = str(df.loc[i,"obs"])
            obs = np.array([float(v) for v in obs.strip()[1:-1].split()])
            

            base_growth_rates[clone][selector] = {"p0g":p0g,
                                                  "p1g":p1g,
                                                  "obs":obs}

    sort_by = []
    for clone in base_growth_rates:
        
        if clone == "wt":
            sort_indexer = (0,-1,-1,clone)
        else:
            these_muts = clone.split("/")
            sort_indexer = [len(these_muts),-1,-1,clone]
            for i in range(len(these_muts)):
                sort_indexer[i+1] = int(these_muts[i][1:-1])

        sort_by.append(tuple(sort_indexer))

    sort_by.sort()

    out_dict = {"genotype":[],
                "selector":[],
                "p0g":[],
                "p1g":[]}
    
    for g in tqdm(sort_by):
        genotype = g[-1]
        for selector in ["kan","pheS"]:
            out_dict["genotype"].append(genotype)
            out_dict["selector"].append(selector)
            for p in ["p0g","p1g"]:
                out_dict[p].append(base_growth_rates[genotype][selector][p])
            
            if len(out_dict) == 4:
                num_values = len(base_growth_rates[genotype][selector]["obs"])
                for i in range(num_values):
                    out_dict[f"obs_{i}"] = []
            for i in range(len(base_growth_rates[genotype][selector]["obs"])):
                out_dict[f"obs_{i}"].append(base_growth_rates[genotype][selector]["obs"][i])
            
            
    return pd.DataFrame(out_dict)

        
base_growth_rates = get_base_growth_rates("many-state_wider/")

        

In [ ]:

import dataprob
import copy

def hill_model(lnK,n,lower,upper,x):

    x = x**n
    K = (np.exp(lnK))**n
    shift = upper - lower

    return lower + shift*(x/(K + x))

fit_parameters = {"lnK":  {"guess":1},
                  "n":    {"guess":1,
                           "lower_bound":-10,
                           "upper_bound":10},
                  "lower":{"guess":0},
                  "upper":{"guess":1}}

colors = {"kan":"darkorange",
          "pheS":"darkgreen"}


def fit_dataset(g,growth_df,base_growth_rates,fig=None,ax=None):

    g_df = growth_df.loc[growth_df["genotype"] == g,:]
    g_bgr_df = base_growth_rates.loc[base_growth_rates["genotype"] == g,:]

    all_x = []
    all_y = []
    all_yerr = []

    fit_results = {}
                   
    for s in pd.unique(g_df["selector"]):
        
        gs_df = g_df.loc[g_df["selector"] == s,:]
        gs_bgr_df = g_bgr_df.loc[g_bgr_df["selector"] == s,:]

        m = np.array(gs_df["slope"])
        m_err = np.array(gs_df["slope_se"])

        if s == "kan":
            min_base = np.array(gs_bgr_df["p0g"])[0]
            max_base = np.array(gs_bgr_df["p1g"])[0]
            normalized = (m - min_base)/(max_base - min_base)
            normalized_se = (m_err - min_base)/(max_base - min_base)
        else:
            min_base = np.array(gs_bgr_df["p0g"])[0]
            max_base = np.array(gs_bgr_df["p1g"])[0]
            normalized = (m - min_base)/(max_base - min_base)
            normalized_se = (m_err - max_base)/(min_base - max_base)

        normalized_se[np.isnan(normalized_se)] = np.abs(max_base - min_base)*0.1
        normalized_se = np.abs(normalized_se)
        
        x = np.array(gs_df["iptg"])

        m = dataprob.setup(hill_model,
                           fit_parameters=copy.deepcopy(fit_parameters),
                           non_fit_kwargs={"x":x})
        m.fit(y_obs=normalized,
              y_std=normalized_se)

        fit_results[s] = m.fit_df.copy()
        
        if ax is not None:
            
            ax.scatter(x,normalized,s=100,fc=colors[s])
            ax.errorbar(x=x,
                        y=normalized,
                        yerr=normalized_se,
                        color=colors[s],lw=0,elinewidth=1,capsize=5)
            ax.plot(x,m.model(m.fit_df["estimate"]),color=colors[s],lw=3)

        all_x.extend(list(x))
        all_y.extend(list(normalized))
        all_yerr.extend(list(normalized_se))

    all_x = np.array(all_x)
    idx = np.argsort(all_x)
    all_x = all_x[idx]
    
    m = dataprob.setup(hill_model,
                       fit_parameters=copy.deepcopy(fit_parameters),
                       non_fit_kwargs={"x":all_x})
    
    m.fit(y_obs=np.array(all_y)[idx],
          y_std=np.array(all_yerr)[idx])
    
    fit_results["all"] = m.fit_df.copy()

    if ax is not None:
        ax.plot(all_x,m.model(m.fit_df["estimate"]),color="skyblue",lw=3)

    
    obs_keys = np.array([f"obs_{i}" for i in range(8)])
    y = np.array(gs_bgr_df.loc[:,obs_keys]).flatten()*12
    m = dataprob.setup(hill_model,
                       fit_parameters=copy.deepcopy(fit_parameters),
                       non_fit_kwargs={"x":x})
    
    m.fit(y_obs=y,y_std=np.ones(len(y))*0.01)
    fit_results["input"] = m.fit_df.copy()

    input_fit = m.fit_df.copy()

    if ax is not None:
        
        ax.plot(x,y,color="blue",lw=3)
    
        ax.set_title(g)
        ax.set_xscale("log")
        plt.show()

    return fit_results



out_dict = {}
for key in ["genotype","fit_on","lnK","lnK_std","n","n_std","lower","lower_std","upper","upper_std"]:
    out_dict[key] = []

counter = 0
for g in tqdm(pd.unique(growth_df["genotype"])):

    counter += 1
    if counter > 200:
         break

    fig, ax = plt.subplots(1,figsize=(6,6))
    #fig, ax = None, None
    
    fit_results = fit_dataset(g,
                              growth_df,
                              base_growth_rates,
                              fig=fig,
                              ax=ax)

    break
    for fit_on in ["kan","pheS","all","input"]:

        lnK, lnK_std, n, n_std, lower, lower_std, upper, upper_std = np.array(fit_results[fit_on].loc[["lnK","n","lower","upper"],["estimate","std"]]).ravel()

        out_dict["genotype"].append(g)
        out_dict["fit_on"].append(fit_on)
        out_dict["lnK"].append(lnK)
        out_dict["lnK_std"].append(lnK_std)
        out_dict["n"].append(n)
        out_dict["n_std"].append(n_std)
        out_dict["lower"].append(lower)
        out_dict["lower_std"].append(lower_std)
        out_dict["upper"].append(upper)
        out_dict["upper_std"].append(upper_std)

    
    


In [ ]:
results_df = pd.DataFrame(out_dict)
input_df = results_df.loc[results_df["fit_on"] == "input",:]
all_df = results_df.loc[results_df["fit_on"] == "all",:]

check_estimate = "lower"

fig, ax = plt.subplots(1,figsize=(6,6))

ax.errorbar(x=input_df[check_estimate],
            #xerr=input_df[f"{check_estimate}_std"],
            y=all_df[check_estimate],
            #yerr=all_df[f"{check_estimate}_std"],
            
            lw=0,capsize=5,elinewidth=1,ms=5,marker="o",markerfacecolor="none")

ax.set_xlim(-5,5)
ax.set_ylim(-5,5)

In [ ]:

def fit_series(y_obs,y_err,m):

    m.fit(y_obs=y_obs,
          y_std=y_err)

    return m.fit_df

def get_fitnesses(growth_df):

    selectors = np.unique(growth_df["selector"])
    selectors.sort()

    guesses = {}
    for s in selectors:
        slopes = np.array(df.loc[df["selector"] == s,"slope"])
        upper = np.mean(slopes[slopes > 0])
        lower = np.mean(slopes[slopes <= 0])



    upper_guess = np.mean(growth_df.loc[growth_df["slope"] > 0,"slope"])
    lower_guess = np.mean(growth_df.loc[growth_df["slope"] <= 0,"slope"])

get_fitnesses(growth_df)

x = np.arange(0,100,10)
y = hill_model(np.log(20),1,lower=lower,upper=upper,x=x)
y_obs = y + np.random.normal(0,0.01,len(y))
y_err = np.ones(len(y_obs))*0.01



fit_series(y_obs,y_err,m=m)




# plt.plot(x,y)
# m.fit_df

In [ ]:
#Used to get results_fitness_full from screen
#There are two options one function does this vectorized which is quicker but hard on memory the other chunks the data taking longer but this revents crashing 
# Usage:
# results_df = calculate_fitness_smart(df) ->this asks how much memory does this machine have to decide which method to use 
# Or explicitly choose one:
# results_df = calculate_fitness_vectorized(df)  # Fastest, needs more memory <-Nat's laptop can do this 
# results_df = calculate_fitness_chunked(df)     # Slower, memory efficient
def calculate_fitness_vectorized(df):
    """
    Vectorized fitness calculation for millions of mutants
    Much faster than nested loops approach
    Dataframe should have df.columns: 'genotype', 'selector', 'iptg', 'time', 'count', 'freq','cfu_per_mL', 'ln_cfu'
    """
    
    print("Starting vectorized fitness calculation...")
    start_time = time.time()
    
    # Group by the combination we need to fit
    grouped = df.groupby(['genotype', 'selector', 'iptg'])
    
    # Pre-allocate lists for results (more efficient than appending)
    n_groups = len(grouped)
    results = {
        'seq': [],
        'iptg': [],
        'selection_condition': [],
        'fitness': [],
        'residual': [],
        'r_squared': [],
        'n_points': []
    }
    
    print(f"Processing {n_groups} unique combinations...")
    
    # Process all groups
    group_count = 0
    for (genotype, selector, iptg), group_df in grouped: #creates subsets of group_df with matching (genotype, selector, iptg) combos
        if len(group_df) < 2:  # Need at least 2 points for linear fit
            continue
            
        # Extract time and ln_cfu as numpy arrays (faster than multiple indexing)
        time_vals = group_df["time"].values
        ln_cfu_vals = group_df["ln_cfu"].values
        
        # Use scipy.stats.linregress - faster than np.polynomial.Polynomial.fit
        slope, intercept, r_value, p_value, std_err = stats.linregress(time_vals, ln_cfu_vals)
        
        # Calculate residual sum of squares manually (faster than full=True)
        predicted = slope * time_vals + intercept
        residual_ss = np.sum((ln_cfu_vals - predicted) ** 2)
        
        # Store results
        results['seq'].append(genotype)
        results['iptg'].append(iptg)
        results['selection_condition'].append(selector)
        results['fitness'].append(slope)
        results['residual'].append(residual_ss)
        results['r_squared'].append(r_value ** 2)
        results['n_points'].append(len(group_df))
        
        group_count += 1
        if group_count % 50000 == 0:  # Progress indicator
            print(f"Processed {group_count:,} / {n_groups:,} combinations...")
    
    end_time = time.time()
    print(f"Fitness calculation completed in {end_time - start_time:.2f} seconds")
    print(f"Processed {group_count:,} valid combinations")
    
    # Create DataFrame from results (much faster than pd.DataFrame(list_of_dicts))
    results_df = pd.DataFrame(results)
    
    return results_df

def calculate_fitness_chunked(df, chunk_size=100000):
    """
    Memory-efficient version that processes data in chunks
    Use this if you run out of memory with the regular version
    """
    
    print("Starting chunked fitness calculation...")
    start_time = time.time()
    
    # Get unique combinations
    combinations = df.groupby(['genotype', 'selector', 'iptg']).size().reset_index(name='count')
    total_combinations = len(combinations)
    
    all_results = []
    
    # Process in chunks
    for chunk_start in range(0, total_combinations, chunk_size):
        chunk_end = min(chunk_start + chunk_size, total_combinations)
        chunk_combinations = combinations.iloc[chunk_start:chunk_end]
        
        print(f"Processing chunk {chunk_start:,} to {chunk_end:,} / {total_combinations:,}")
        
        chunk_results = {
            'seq': [],
            'iptg': [],
            'selection_condition': [],
            'fitness': [],
            'residual': [],
            'r_squared': [],
            'n_points': []
        }
        
        for _, row in chunk_combinations.iterrows():
            genotype, selector, iptg = row['genotype'], row['selector'], row['iptg']
            
            # Filter data for this combination
            mask = (df['genotype'] == genotype) & (df['selector'] == selector) & (df['iptg'] == iptg)
            group_df = df[mask]
            
            if len(group_df) < 2:
                continue
                
            # Linear regression
            time_vals = group_df["time"].values
            ln_cfu_vals = group_df["ln_cfu"].values
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(time_vals, ln_cfu_vals)
            
            # Calculate residual
            predicted = slope * time_vals + intercept
            residual_ss = np.sum((ln_cfu_vals - predicted) ** 2)
            
            # Store results
            chunk_results['seq'].append(genotype)
            chunk_results['iptg'].append(iptg)
            chunk_results['selection_condition'].append(selector)
            chunk_results['fitness'].append(slope)
            chunk_results['residual'].append(residual_ss)
            chunk_results['r_squared'].append(r_value ** 2)
            chunk_results['n_points'].append(len(group_df))
        
        # Convert chunk to DataFrame and store
        chunk_df = pd.DataFrame(chunk_results)
        all_results.append(chunk_df)
    
    # Combine all chunks
    results_df = pd.concat(all_results, ignore_index=True)
    
    end_time = time.time()
    print(f"Chunked calculation completed in {end_time - start_time:.2f} seconds")
    print(f"Total results: {len(results_df):,}")
    
    return results_df

# Choose the appropriate function based on your memory constraints
def calculate_fitness_smart(df, memory_limit_gb=8):
    """
    Automatically choose between vectorized and chunked approach based on data size
    """
    
    # Estimate memory usage (rough calculation)
    estimated_memory_gb = len(df) * len(df.columns) * 8 / (1024**3)  # 8 bytes per float64
    
    print(f"Estimated memory usage: {estimated_memory_gb:.2f} GB")
    
    if estimated_memory_gb > memory_limit_gb:
        print("Using chunked approach due to memory constraints...")
        return calculate_fitness_chunked(df)
    else:
        print("Using vectorized approach...")
        return calculate_fitness_vectorized(df)



print("Functions defined. Usage:")
print("results_df = calculate_fitness_smart(df)  # Automatic choice")
print("results_df = calculate_fitness_vectorized(df)  # Fastest")  
print("results_df = calculate_fitness_chunked(df)  # Memory efficient")
#done

In [ ]:
#Read in csv file with seqs and cfu/ml at each iptg conc. 
#This dataframe has the headers 'Unnamed: 0', 'genotype', 'selector', 'iptg', 'time', 'count', 'freq','cfu_per_mL'
df_start = pd.read_csv("2025-05-17_simulate-sampling\output\output_obs_20250703_NJ_df.csv")
#done
df_start = df_start.drop(columns=["Unnamed: 0"])#remove the columns that is just the line number 
#done
df_start["ln_cfu"] = np.log2(df_start["cfu_per_mL"]) #take the log base 2 of the cfu_per_mL - seems to be common
#done


In [ ]:
results_df = calculate_fitness_smart(df_start)#this calculates the fitness/growth rate from a line with more than 2 ln cfu per ml over time
#returns as df with columns 'seq', 'iptg', 'selection_condition', 'fitness', 'residual', 'r_squared', 'n_points'

In [ ]:
results_df.to_csv("results_fitness_full.csv")